In [1]:
"""
module neuron #(
    parameter N = 2,       // Number of inputs -> 
    parameter WIDTH = 16,    // Bit width of inputs/weights/bias
    parameter biasFile="",
    parameter weightFile=""
)
(
    input clk,
    input reset,
    input   [N*WIDTH-1:0]  x_flat,       // Flattened inputs
    output reg  [(2*WIDTH)-1:0]    y             // Output
    );
    // Internal register to hold the sum
    reg signed [(2*WIDTH)-1:0] sum;
    reg signed [WIDTH-1:0] bb [0:0];
    reg signed [WIDTH-1:0] weight_mem [0:N-1];
    reg signed [WIDTH-1:0] x [0:N-1];
    
    genvar i;
    reg [255:0] filename;
    initial begin
            $readmemh(biasFile, bb, 0, 0);
            $readmemh(weightFile, weight_mem);
    end
    
    generate
        for (i = 0; i < N; i = i + 1) begin : unpack_x_w
            always @(*) begin
                x[i] = x_flat[(i+1)*WIDTH-1 : i*WIDTH];  // Unpack x[i]
            end
        end
    endgenerate
    integer j;
    always @(*) begin
            sum = bb[0];
            // Compute weighted sum
            for (j = 0; j < N; j = j + 1) begin
                sum = sum + (x[j] * weight_mem[j]);  // Multiply x[i] with w[i] and accumulate
            end
            // ReLU activation
            if (sum < 0)
                y <= 0;
            else
                y <= sum; // Truncate to WIDTH bits
    end
endmodule




`timescale 1ns / 1ps
module layer2 #(
    parameter N = 2,               // Number of inputs per neuron   ->  no of input
    parameter WIDTH = 16,           // Bit width of inputs/weights/bias
    parameter NUM_NEURONS = 4,       // Number of neurons in this layer
    parameter layer_num = 2          // layer number
)
( 
    input clk,
    input reset,
    output [(NUM_NEURONS*2*WIDTH)-1:0] y_flat            // Outputs from all neurons
    );
    reg [N*WIDTH-1:0] x; 

    initial begin
        x = 32'h00490020;   // x1 = 0020, x2 = 0049
    end    
    // Instantiate neurons
    genvar n;
    generate
        for (n = 0; n < NUM_NEURONS; n = n + 1) begin : neuron_inst
            neuron #(
                .N(N),
                .WIDTH(WIDTH),
                .biasFile({"b_", layer_num[N:0] + "0" ,"_", n[N:0] + "0", ".mif"}),
                .weightFile({"w_2_",n[N:0] + "0", ".mif"})
            ) u_neuron (
                .clk(clk),
                .reset(reset),
                .x_flat(x),    // Flattened input remains the same for each neuron
                .y(y_flat[(n*2*WIDTH) +: 2*WIDTH])            // Output of this neuron
            );
        end
    endgenerate    
endmodule




`timescale 1ns / 1ps

module layer3 #(
    parameter N = 4,               // Number of inputs per neuron   ->  no of input
    parameter WIDTH = 16,           // Bit width of inputs/weights/bias
    parameter NUM_NEURONS = 2,       // Number of neurons in this layer
    parameter layer_num = 3
)
(
    input clk,
    input reset,
    input  [2*N*WIDTH-1:0]    x,
    output [4*NUM_NEURONS*WIDTH-1:0] y_flat
    );
    genvar n;
    integer i;
    generate
        for (n = 0; n < NUM_NEURONS; n = n + 1) begin : neuron_inst
            neuron #(
                .N(N),
                .WIDTH(2*WIDTH),
                .biasFile({"b_3_", n[N:0] + "0", ".mif"}),
                .weightFile({"w_3_",n[N:0] + "0", ".mif"})
            ) u_neuron (
                .clk(clk),
                .reset(reset),
                .x_flat(x),    // Flattened input remains the same for each neuron),
                .y(y_flat[(4*n*WIDTH) +: 4*WIDTH])            // Output of this neuron
            );
        end
    endgenerate
endmodule

"""
from pynq import Overlay , MMIO
overlay = Overlay('two.bit')
gpio_0 = MMIO(0x41200000,0x10000)
result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x1656a
result : 0b10110010101101010


In [1]:
"""
`timescale 1ns / 1ps

module neuron #(
    parameter N = 2,       // Number of inputs -> 
    parameter WIDTH = 16,    // Bit width of inputs/weights/bias
    parameter biasFile="",
    parameter weightFile=""
)
(
    input clk,
    input reset,
    input   [N*WIDTH-1:0]  x_flat,       // Flattened inputs
    output reg  [(2*WIDTH)-1:0]    y             // Output
    );
    // Internal register to hold the sum
    reg signed [(2*WIDTH)-1:0] sum;
    reg signed [WIDTH-1:0] bb [0:0];
    reg signed [WIDTH-1:0] weight_mem [0:N-1];
    reg signed [WIDTH-1:0] x [0:N-1];
    
    genvar i;
    reg [255:0] filename;
    initial begin
            $readmemh(biasFile, bb, 0, 0);
            $readmemh(weightFile, weight_mem);
    end
    
    generate
        for (i = 0; i < N; i = i + 1) begin : unpack_x_w
            always @(*) begin
                x[i] = x_flat[(i+1)*WIDTH-1 : i*WIDTH];  // Unpack x[i]
            end
        end
    endgenerate
    integer j;
    always @(*) begin
            sum = bb[0];
            // Compute weighted sum
            for (j = 0; j < N; j = j + 1) begin
                sum = sum + (x[j] * weight_mem[j]);  // Multiply x[i] with w[i] and accumulate
            end
            // ReLU activation
            if (sum < 0)
                y <= 0;
            else
                y <= sum; // Truncate to WIDTH bits
    end
endmodule

"""
from pynq import Overlay , MMIO
overlay = Overlay('one.bit')
gpio_0 = MMIO(0x41200000,0x10000)
result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x37e0656a
result : 0b110111111000000110010101101010


In [3]:
"""
module neu #(
    parameter N = 2,       // Number of inputs -> 
    parameter WIDTH = 16,   // Bit width of inputs/weights/bias
    parameter biasFile="",
    parameter weightFile=""
)
(
    input clk,
    input reset,
    input   [N*WIDTH-1:0]  x_flat,       // Flattened inputs
    input   [N*WIDTH-1:0]  w_flat,       // Flattened weights
    input   [WIDTH-1:0]    b,            // Bias
    input [N:0] layer_num,
    input [N:0] neuron_num,
    output reg  [WIDTH-1:0]    y             // Output
    );

    // Internal register to hold the sum
    reg signed [31:0] sum;
    reg signed [WIDTH-1:0] bb [0:0];
    
    // Declare registers to store individual x and w values
    reg signed [WIDTH-1:0] x [0:N-1];
    reg signed [WIDTH-1:0] w [0:N-1];
    reg signed [WIDTH-1:0] weight_mem [0:N-1];
    
    genvar i;
    integer j;
    reg [255:0] filename;
    initial begin
            $readmemh(biasFile, bb, 0, 0);
            $readmemh(weightFile, weight_mem);
    end
    generate
        for (i = 0; i < N; i = i + 1) begin : unpack_x_w
            always @(*) begin
                x[i] = x_flat[(i+1)*WIDTH-1 : i*WIDTH];  // Unpack x[i]
                w[i] = w_flat[(i+1)*WIDTH-1 : i*WIDTH];  // Unpack w[i]
            end
        end
    endgenerate

    always @(*) begin
            sum = bb[0];
            for (j = 0; j < N; j = j + 1) begin
                sum = sum + (x[j] * weight_mem[j]);  // Multiply x[i] with w[i] and accumulate
            end
//             ReLU activation
            if (sum < 0)
                y <= 0;
            else
                y <= sum[WIDTH-1:0]; // Truncate to WIDTH bits
    end
endmodule






module two #(
    parameter N = 2,               // Number of inputs per neuron   ->  no of input
    parameter WIDTH = 16,           // Bit width of inputs/weights/bias
    parameter NUM_NEURONS = 4,       // Number of neurons in this layer
    parameter layer_num = 2          // layer number
)
( 
    input clk,
    input reset,
    output [NUM_NEURONS*WIDTH-1:0] y_flat            // Outputs from all neurons
    );
//  Internal registers to store unpacked weights, biases, and outputs
    reg signed [WIDTH-1:0] b [0:NUM_NEURONS-1];           // 1D array for biases
    reg [WIDTH-1:0] y [0:NUM_NEURONS-1];                 // Wire array for neuron outputs
    reg signed [NUM_NEURONS*N*WIDTH-1:0] w;
    reg signed [WIDTH-1:0] weight_mem [0:N-1];  // temporary memory to hold weights from file
    reg [N*WIDTH-1:0] x;
    reg [15:0] mem [0:15]; // A memory array
    
    genvar n;
    integer i;
    integer j;
    reg [255:0] filename;

    // bias initialization
    initial begin         
        x = 32'h00490020;   // x1 = 0020, x2 = 0049
        w = 128'h00190020002100220023002400250026;   // w1 = 0026,  w2 = 0025,  w3 = 0024, w4 == 0023, w5 = 0022, w6 = 0021 w7 = 

    end    

//    // Instantiate neurons
    generate
        for (n = 0; n < NUM_NEURONS; n = n + 1) begin : neuron_inst
            neu #(
                .N(N),
                .WIDTH(WIDTH),
                .biasFile({"b_", layer_num[N:0] + "0" ,"_", n[N:0] + "0", ".mif"}),
                .weightFile({"w_2_",n[N:0] + "0", ".mif"})
            ) u_neuron (
                .clk(clk),
                .reset(reset),
                .x_flat(x),    // Flattened input remains the same for each neuron
                .w_flat(w[(n*N + 0) * WIDTH +: N*WIDTH]),    // Flattened weights remain the same
                .b(b[n]),           // Pass the corresponding bias for this neuron
                .layer_num(layer_num),
                .neuron_num(n),
                .y(y_flat[(n*WIDTH) +: WIDTH])            // Output of this neuron
            );
        end
    endgenerate
endmodule








module three #(
    parameter N = 4,               // Number of inputs per neuron   ->  no of input
    parameter WIDTH = 16,           // Bit width of inputs/weights/bias
    parameter NUM_NEURONS = 2,       // Number of neurons in this layer
    parameter layer_num = 3
)

(
    input clk,
    input reset,
    input  [N*WIDTH-1:0]    x,
    output [NUM_NEURONS*WIDTH-1:0] y_flat
    );
    
    reg signed [WIDTH-1:0] b [0:NUM_NEURONS-1];           // 1D array for biases
    reg [NUM_NEURONS*N*WIDTH-1:0] w;
    genvar n;
    integer i;
    
    initial begin
        w = 128'h00120011001000090008000700060005;   // w1 = 0026,  w2 = 0025,  w3 = 0024, w4 == 0023, w5 = 0022, w6 = 0021
    end    
    
    generate
        for (n = 0; n < NUM_NEURONS; n = n + 1) begin : neuron_inst
            neu #(
                .N(N),
                .WIDTH(WIDTH),
                .biasFile({"b_3_", n[N:0] + "0", ".mif"}),
                .weightFile({"w_3_",n[N:0] + "0", ".mif"})
            ) u_neuron (
                .clk(clk),
                .reset(reset),
                .x_flat(x),    // Flattened input remains the same for each neuron
                .w_flat(w[(n*N + 0) * WIDTH +: N*WIDTH]),    // Flattened weights remain the same
                .b(b[n]),           // Pass the corresponding bias for this neuron
                .layer_num(layer_num),
                .neuron_num(n),
                .y(y_flat[(n*WIDTH) +: WIDTH])            // Output of this neuron
            );
        end
    endgenerate
    
endmodule


"""
from pynq import Overlay , MMIO

overlay = Overlay('two.bit')

gpio_0_baseaddr = 0x41200000

gpio_0 = MMIO(0x41200000,0x10000)

result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

# giving result


KeyboardInterrupt



In [3]:
"""
`timescale 1ns / 1ps
module neu #(
    parameter N = 2,       // Number of inputs -> 
    parameter WIDTH = 16    // Bit width of inputs/weights/bias
)
(
    input clk,
    input reset,
    input   [N*WIDTH-1:0]  x_flat,       // Flattened inputs
    input   [N*WIDTH-1:0]  w_flat,       // Flattened weights
    input   [WIDTH-1:0]    b,            // Bias
    input [N:0] layer_num,
    input [N:0] neuron_num,
    output reg  [WIDTH-1:0]    y             // Output
    );
    reg signed [31:0] sum;
    reg signed [WIDTH-1:0] bb [0:0];
    reg signed [WIDTH-1:0] x [0:N-1];
    reg signed [WIDTH-1:0] w [0:N-1];
    
    genvar i;
    integer j;
    reg [255:0] filename;
    
    initial begin
        filename = {"b_", layer_num[N:0] + "0" ,"_", neuron_num[N:0] + "0", ".mif"};
        $readmemh("b_2_0.mif", bb, 0, 0);
    end
    
    generate
        for (i = 0; i < N; i = i + 1) begin : unpack_x_w
            always @(*) begin
                x[i] = x_flat[(i+1)*WIDTH-1 : i*WIDTH];  // Unpack x[i]
                w[i] = w_flat[(i+1)*WIDTH-1 : i*WIDTH];  // Unpack w[i]
            end
        end
    endgenerate

    always @(*) begin
            sum = bb[0];
            for (j = 0; j < N; j = j + 1) begin
                sum = sum + (x[j] * w[j]);  // Multiply x[i] with w[i] and accumulate
            end
            if (sum < 0)
                y <= 0;
            else
                y <= sum[WIDTH-1:0]; // Truncate to WIDTH bits
    end
endmodule



`timescale 1ns / 1ps
module two #(
    parameter N = 2,               // Number of inputs per neuron   ->  no of input
    parameter WIDTH = 16,           // Bit width of inputs/weights/bias
    parameter NUM_NEURONS = 4,       // Number of neurons in this layer
    parameter layer_num = 2          // layer number
)
( 
    input clk,
    input reset,
    output [NUM_NEURONS*WIDTH-1:0] y_flat            // Outputs from all neurons
    );
    reg signed [WIDTH-1:0] b [0:NUM_NEURONS-1];           // 1D array for biases
    reg [WIDTH-1:0] y [0:NUM_NEURONS-1];                 // Wire array for neuron outputs
    reg signed [NUM_NEURONS*N*WIDTH-1:0] w;
    reg signed [WIDTH-1:0] weight_mem [0:N-1];  // temporary memory to hold weights from file
    reg [N*WIDTH-1:0] x;
    reg [15:0] mem [0:15]; // A memory array
    
    genvar n;
    integer i;
    integer j;
    reg [255:0] filename;
    initial begin         
        x = 32'h00490020;   // x1 = 0020, x2 = 0049
        w = 128'h00190020002100220023002400250026;   // w1 = 0026,  w2 = 0025,  w3 = 0024, w4 == 0023, w5 = 0022, w6 = 0021 w7 = 
    end    

    generate
        for (n = 0; n < NUM_NEURONS; n = n + 1) begin : neuron_inst
            neu #(
                .N(N),
                .WIDTH(WIDTH)
            ) u_neuron (
                .clk(clk),
                .reset(reset),
                .x_flat(x),    // Flattened input remains the same for each neuron
                .w_flat(w[(n*N + 0) * WIDTH +: N*WIDTH]),    // Flattened weights remain the same
                .b(b[n]),           // Pass the corresponding bias for this neuron
                .layer_num(layer_num),
                .neuron_num(n),
                .y(y_flat[(n*WIDTH) +: WIDTH])            // Output of this neuron
            );
        end
    endgenerate
endmodule




`timescale 1ns / 1ps
module three #(
    parameter N = 4,               // Number of inputs per neuron   ->  no of input
    parameter WIDTH = 16,           // Bit width of inputs/weights/bias
    parameter NUM_NEURONS = 2,       // Number of neurons in this layer
    parameter layer_num = 3
)
(
    input clk,
    input reset,
    input  [N*WIDTH-1:0]    x,
    output [NUM_NEURONS*WIDTH-1:0] y_flat
    );
    
    reg signed [WIDTH-1:0] b [0:NUM_NEURONS-1];           // 1D array for biases
    reg [NUM_NEURONS*N*WIDTH-1:0] w;
    genvar n;
    integer i;
    
    initial begin
        w = 128'h00120011001000090008000700060005;   // w1 = 0026,  w2 = 0025,  w3 = 0024, w4 == 0023, w5 = 0022, w6 = 0021
    end    
    
    generate
        for (n = 0; n < NUM_NEURONS; n = n + 1) begin : neuron_inst
            neu #(
                .N(N),
                .WIDTH(WIDTH)
            ) u_neuron (
                .clk(clk),
                .reset(reset),
                .x_flat(x),    // Flattened input remains the same for each neuron
                .w_flat(w[(n*N + 0) * WIDTH +: N*WIDTH]),    // Flattened weights remain the same
                .b(b[n]),           // Pass the corresponding bias for this neuron
                .layer_num(layer_num),
                .neuron_num(n),
                .y(y_flat[(n*WIDTH) +: WIDTH])            // Output of this neuron
            );
        end
    endgenerate
    
endmodule
"""

from pynq import Overlay , MMIO

overlay = Overlay('three.bit')

gpio_0_baseaddr = 0x41200000

gpio_0 = MMIO(gpio_0_baseaddr,0x10000)

result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x3500647a
result : 0b110101000000000110010001111010


In [6]:
"""
           $readmemh("data.mif", mem);
           for(j=0; j < NUM_NEURONS*N; j= j+1) begin
                w[j*WIDTH +: WIDTH] = mem[(NUM_NEURONS * N -1) - j];
           end
"""
from pynq import Overlay , MMIO

overlay = Overlay('twentyfive.bit')

gpio_0_baseaddr = 0x41200000

gpio_0 = MMIO(gpio_0_baseaddr,0x10000)

result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x15f00960
result : 0b10101111100000000100101100000


In [5]:
"""
        $readmemh("w_2_0.mif", weight_mem);
        w[31:0] = {weight_mem[1], weight_mem[0]};
        
        $readmemh("w_2_1.mif", weight_mem);
        w[63:32] = {weight_mem[1], weight_mem[0]};
        
        $readmemh("w_2_2.mif", weight_mem);
        w[95:64] = {weight_mem[1], weight_mem[0]};
        
        $readmemh("w_2_3.mif", weight_mem);
        w[127:96] = {weight_mem[1], weight_mem[0]};
"""
from pynq import Overlay , MMIO

overlay = Overlay('twentyfour.bit')

gpio_0_baseaddr = 0x41200000

gpio_0 = MMIO(gpio_0_baseaddr,0x10000)

result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x15f00960
result : 0b10101111100000000100101100000


In [4]:
"""
            w[31:0] = {16'h0025, 16'h0026};
            w[63:32] = {16'h0023, 16'h0024};
            w[95:64] = {16'h0021, 16'h0022};
            w[127:96] = {16'h0019, 16'h0020};
"""
from pynq import Overlay , MMIO

overlay = Overlay('twentythree.bit')

gpio_0_baseaddr = 0x41200000

gpio_0 = MMIO(gpio_0_baseaddr,0x10000)

result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x37e0656a
result : 0b110111111000000110010101101010


In [3]:
"""
            $readmemh("data.mif", mem);
           for(j=0; j < NUM_NEURONS*N; j= j+1) begin
                w[j*WIDTH +: WIDTH] = mem[j];
           end
"""
from pynq import Overlay , MMIO

overlay = Overlay('twentytwo.bit')

gpio_0_baseaddr = 0x41200000

gpio_0 = MMIO(gpio_0_baseaddr,0x10000)

result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x15f00960
result : 0b10101111100000000100101100000


In [2]:
"""
        w[31:0] = {16'h0026, 16'h0025};
        w[63:32] = {16'h0024, 16'h0023};
        w[95:64] = {16'h0022, 16'h0021};
        w[127:96] = {16'h0020, 16'h0019};
"""
from pynq import Overlay , MMIO

overlay = Overlay('twentyone.bit')

gpio_0_baseaddr = 0x41200000

gpio_0 = MMIO(gpio_0_baseaddr,0x10000)

result  = gpio_0.read(0x00)
print(f"result : {hex(result)}")
print(f"result : {bin(result)}")

result : 0x52c87144
result : 0b1010010110010000111000101000100
